# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [187]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 

In [163]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [12]:
def fit_generative_model(x,y, features):
    k = len(np.unique(y))  # labels 1,2,...,k
    d = len(features)  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[np.ix_(indices, features)], axis = 0)
        sigma[label] = np.cov(x[np.ix_(indices, features)], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [17]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx, trainy, [0,2,6])
print(mu, sigma, pi, sep = '\n')

[[ 0.          0.          0.        ]
 [13.78534884  2.42790698  2.99627907]
 [12.31092593  2.22703704  2.10907407]
 [13.15969697  2.40090909  0.75727273]]
[[[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.23325279 -0.00393532  0.07526874]
  [-0.00393532  0.03677469 -0.00140779]
  [ 0.07526874 -0.00140779  0.15240941]]

 [[ 0.2819047  -0.03746578 -0.03911211]
  [-0.03746578  0.11230233  0.09116207]
  [-0.03911211  0.09116207  0.56869729]]

 [[ 0.2851787   0.01545785  0.01006887]
  [ 0.01545785  0.02675978  0.02115399]
  [ 0.01006887  0.02115399  0.07375317]]]
[0.         0.33076923 0.41538462 0.25384615]


<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [23]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    if len(features) > 1:
        if features[0] == features[1]: # need f1 != f2
            print("Please choose different features for f1 and f2.")
            return  
        
    mu, sigma, pi = fit_generative_model(tx, ty, features)
    
    k = len(np.unique(ty))
    nt = len(ty) # Number of test points
    score = np.zeros((nt,k+1))
    for i in range(0,nt):
        for label in range(1,k+1):
            score[i,label] = np.log(pi[label]) + \
            multivariate_normal.logpdf(testx[i,features], mean=mu[label,:], cov=sigma[label,:,:])
    predictions = np.argmax(score[:,1: k+1], axis=1) + 1
    # Finally, tally up score
    errors = np.sum(predictions != testy)
    print("Test error using feature(s): ")
    for f in features:
        print("'" + featurenames[f] + "'" + " ")
    print("Errors: " + str(errors) + "/" + str(nt))# Now test the performance of a predictor based on a subset of features
    
    return errors

### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [25]:
errors = test_model(mu, sigma, pi, [2], testx, testy)

Test error using feature(s): 
'Ash' 
Errors: 27/48


Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [110]:
import itertools

k = len(np.unique(trainy))
errors = 1e6 * np.ones((k+1,1))
min_err = {'val':[1e4], 'arg':[]}
    
for i in range(k):
    for j in range(i, k):
        if i != j:
            errors[i + j] = test_model(mu, sigma, pi, [i, j], testx, testy)
        
            if np.min(errors) < min_err['val']:
                min_err['val'] = np.min(errors)
                min_err['arg'].append([i, j])
                
print('\n\n\nMin. error for'+ '\n' + featurenames[min_err['arg'][0][0]] + ' and ' + featurenames[min_err['arg'][0][1]]) 
print('\n' + 'Incorrect classifications = ' + str(min_err['val']) )

Test error using feature(s): 
'Alcohol' 
'Malic acid' 
Errors: 8/48
Test error using feature(s): 
'Alcohol' 
'Ash' 
Errors: 11/48
Test error using feature(s): 
'Malic acid' 
'Ash' 
Errors: 16/48



Min. error for
Alcohol and Malic acid

Incorrect classifications = 8.0


Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [21]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

Test error using feature(s): 
'Alcohol' 
'Ash' 
'Flavanoids' 
Errors: 2/48


Exercise 4. How many errors when using all 13 features?

In [22]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

Test error using feature(s): 
'Alcohol' 
'Malic acid' 
'Ash' 
'Alcalinity of ash' 
'Magnesium' 
'Total phenols' 
'Flavanoids' 
'Nonflavanoid phenols' 
'Proanthocyanins' 
'Color intensity' 
'Hue' 
'OD280/OD315 of diluted wines' 
'Proline' 
Errors: 0/48


## Naive Bayes

In [11]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm



def load_and_process_data(filename):
    # Load data set.
    data = np.loadtxt(filename, delimiter=',')
    # Names of features
    featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
    # Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
    np.random.seed(0)
    perm = np.random.permutation(178)
    trainx = data[perm[0:130],1:14]
    trainy = data[perm[0:130],0]
    testx = data[perm[130:178], 1:14]
    testy = data[perm[130:178],0]
    
    return trainx, trainy, testx, testy, featurenames




def fit_naive_generative_model(trainx, trainy, features):
    n_class = len(np.unique(trainy))
    n_feat = len(features)
    
    mu = np.zeros((n_class, n_feat))
    var = np.zeros((n_class, n_feat))
    pi = np.zeros((n_class, 1))
    
    for cl in range(n_class):
        for feat in range(n_feat):
            feature = features[feat]
            indices = (trainy == cl+1)
            mu[cl, feat] = np.mean(trainx[indices, feature])
            var[cl, feat] = np.var(trainx[indices, feature])
            
    pi = np.array([np.sum(trainy == 1), np.sum(trainy == 2), np.sum(trainy == 3)]) / float(len(trainy))
    
    return mu, var, pi 




def fit_normal(x, mu, var):
    P = norm.logpdf(x, mu, np.sqrt(var))
    return P
        

    
    
def predict_class(tx, ty, features, mu, var, pi):
    n_feat = len(features)
    n_class = len(np.unique(ty))
    n = tx.shape[0]
    
    P = np.zeros((n, n_class, n_feat))
    Scores = np.zeros((n, n_class))
    
    for i in range(n):
        for cl in range(0, n_class):
            for feat in range(n_feat):
                feature = features[feat]
                P[i, cl, feat] = fit_normal(tx[i, feature], mu[cl, feat], var[cl, feat])
            Scores[i, cl] = np.log(pi[cl]) +  np.sum(P[i, cl, :])
        
    predictions = np.argmax(Scores, axis=1) + 1
    
    return predictions, P, Scores




def accuracy(predictions, ty, features, featurenames):
    errors = np.sum(predictions != ty)
    
    print("Test error using feature(s): ")
    
    for f in features:
        print("'" + featurenames[f] + "'" + " ")
        
    print("Errors: " + str(errors) + "/" + str(len(ty)) + "= " + str(100* ((len(ty) - errors) / len(ty))) + "%")
    

    
    
def NaiveBayes(file, features, output = False):
    """Function to obtain the Naive Bayes Classifier for a dataset contained in a comma separated text file.
    
    Inputs: 
    1. Filename in .txt format (a string)
    2. Features or number of the columns that contain the desired features
    3. Output if true, will display Errors values, Probabilitites and Scores for all data points, classes and desired features
    
    Output:
    If output is True:
    Error, Probabilities for each class and each feature of each prediction, and Scores for each class for each prediction
    
    Default: (also if output is False)
    Featurenames and Associated Accuracy"""
    
    
    
    n_out = 3
    n_in = 3
    
    try:
        if (len(np.unique(features)) != len(features)) or (np.sign(np.prod(features)) == -1):
            print("Please Enter Unique positive numbers for features")
            return [None]*n_out
        else:
            #Load and Process Data
            trainx, trainy, testx, testy, featurenames = load_and_process_data(file)
            
            #Fit generative model
            mu, var, pi = fit_naive_generative_model(trainx, trainy, features)
    
            #Find predictions for test cases
            predictions, P, Scores = predict_class(testx, testy, features, mu, var, pi)
    
            #Find Accuracy
            errors = accuracy(predictions, testy, features, featurenames)
            
            if output:
                return errors, P, Scores
    
    except TypeError:
        print("Please Enter the feature set as a list of numbers or an 1D array of numbers or check the type of data entered. It must be numerical")
        return [None]*n_out
    
    except IndexError:
        print("Please Enter a valid Index between 0 and the number of features in the dataset")
        return [None]*n_out
            

In [12]:
NaiveBayes('wine.data.txt', range(12))

Test error using feature(s): 
'Alcohol' 
'Malic acid' 
'Ash' 
'Alcalinity of ash' 
'Magnesium' 
'Total phenols' 
'Flavanoids' 
'Nonflavanoid phenols' 
'Proanthocyanins' 
'Color intensity' 
'Hue' 
'OD280/OD315 of diluted wines' 
Errors: 2/48= 95.83333333333334%
